In [1]:
import os
import pandas as pd
import numpy as np
import requests
import gzip
import pgeocode
#from isdparser import isdparser
from meteostat import Point, Hourly
#from datetime import datetime
import datetime

"""usaf = "722950"
wban = "23174"
year = "2024"

url = f"https://www.ncei.noaa.gov/pub/data/noaa/{year}/{usaf}-{wban}-{year}.gz"
response = requests.get(url)

with open("station-raw.gz", "wb") as f:
    f.write(response.content)

records = isdparser.ISDParser("station-raw.gz").records
df = pd.DataFrame(records)
df.head()

stations = pd.read_csv("https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv")
# Filter to stations of interest (e.g., by ICAO code like KPHL, KBWI, KCMH etc.)
stations = stations[stations['ICAO'] == 'KCMH']
print(stations)

years = range(2024, 2025)
for y in years:
    url = f"https://www.ncei.noaa.gov/pub/data/noaa/{y}/{usaf}-{wban}-{y}.gz"
    r = requests.get(url)
    open(f"{usaf}-{wban}-{y}.gz", "wb").write(r.content)"""

'usaf = "722950"\nwban = "23174"\nyear = "2024"\n\nurl = f"https://www.ncei.noaa.gov/pub/data/noaa/{year}/{usaf}-{wban}-{year}.gz"\nresponse = requests.get(url)\n\nwith open("station-raw.gz", "wb") as f:\n    f.write(response.content)\n\nrecords = isdparser.ISDParser("station-raw.gz").records\ndf = pd.DataFrame(records)\ndf.head()\n\nstations = pd.read_csv("https://www.ncei.noaa.gov/pub/data/noaa/isd-history.csv")\n# Filter to stations of interest (e.g., by ICAO code like KPHL, KBWI, KCMH etc.)\nstations = stations[stations[\'ICAO\'] == \'KCMH\']\nprint(stations)\n\nyears = range(2024, 2025)\nfor y in years:\n    url = f"https://www.ncei.noaa.gov/pub/data/noaa/{y}/{usaf}-{wban}-{y}.gz"\n    r = requests.get(url)\n    open(f"{usaf}-{wban}-{y}.gz", "wb").write(r.content)'

In [58]:
def getWeatherFilePath(load_area):
    filepath = "data/weather/" + load_area + ".csv"
    return filepath

def getPjmFilePath(year):
    filepath = "data/pjm/" + "hrl_load_metered_" + str(year) + ".csv"
    return filepath

def getCompleteDfFilePath(load_area):
    filepath = "data/complete_dfs/" + load_area + ".csv"
    return filepath

In [3]:
# RTO is the *entire PJM footprint*, not a load zone. no meaningful ZIP.
zone_to_zips = {
    # Atlantic City Electric (Southern New Jersey)
    'AECO': ['08401'],   # Atlantic City, NJ
    
    # American Electric Power - Appalachian Power (central and Southern West Virginia)
    'AEPAPT': ['25301'],   # Charleston, WV

    # American Electric Power - Indiana Michigan Power (northeast quadrant of indiana and southwest corner of michigan)
    'AEPIMP': ['46802'],   # Fort Wayne, IN

    # American Electric Power - Kentucky Power (eastern kentucky)
    'AEPKPT': ['41101'],   # Ashland, KY (Eastern Kentucky Power region)

    # American Electric Power - Ohio (central and southeast ohio)
    'AEPOPT': ['43215'],   # Columbus, OH

    # Allegheny Power (FirstEnergy West) serving MD/WV/PA panhandle
    'AP': ['21502'],     # Cumberland, MD

    # Baltimore Gas & Electric
    'BC': ['21201'],     # Baltimore, MD (city center)

    # Cleveland Electric Illuminating Company (FirstEnergy)
    'CE': ['44114'],     # Cleveland, OH (downtown)

    # Dayton Power & Light (AES Ohio)
    'DAY': ['45402'],    # Dayton, OH

    # Duke Energy Ohio/Kentucky load zone
    'DEOK': ['45202'],   # Cincinnati, OH

    # Dominion Virginia Power
    'DOM': ['23219'],    # Richmond, VA

    # Delmarva Power (Delaware & Eastern Shore MD)
    'DPLCO': ['19901'],  # Dover, DE

    # Duquesne Light
    'DUQ': ['15222'],    # Pittsburgh, PA

    # Easton Utilities (Maryland municipal)
    'EASTON': ['21601'], # Easton, MD

    # East Kentucky Power Cooperative
    'EKPC': ['40391'],   # Winchester, KY

    # Jersey Central Power & Light (FirstEnergy NJ)
    'JC': ['07728'],     # Freehold, NJ

    # Metropolitan Edison (FirstEnergy PA)
    'ME': ['19601'],     # Reading, PA

    # Ohio Edison (FirstEnergy OH)
    'OE': ['44308'],     # Akron, OH

    # Ohio Valley Electric Corporation
    'OVEC': ['45661'],   # Piketon, OH

    # Pennsylvania Power Company (FirstEnergy PA)
    'PAPWR': ['16101'],  # New Castle, PA

    # PECO (Philadelphia)
    'PE': ['19103'],     # Philadelphia, PA (Center City)

    # Potomac Electric Power Company (Washington DC + Montgomery Co MD)
    'PEPCO': ['20001'],  # Washington, DC

    # Potomac Edison (FirstEnergy MD/WV)
    'PLCO': ['21740'],   # Hagerstown, MD

    # Pennsylvania Electric Company (Penelec - FirstEnergy Northwest/Central PA)
    'PN': ['16601'],     # Altoona, PA

    # Public Service Electric & Gas (PSE&G NJ)
    'PS': ['07102'],     # Newark, NJ

    # Rockland Electric (Northern NJ / small NY portion)
    'RECO': ['07450'],   # Ridgewood, NJ

    # Southern Maryland Electric Cooperative
    'SMECO': ['20650'],  # Leonardtown, MD

    # UGI Electric (NE Pennsylvania, Luzerne County)
    'UGI': ['18702'],    # Wilkes-Barre, PA

    # VMEU = Virginia Municipal Electric Utility (multiple small cities)
    'VMEU': ['22801']    # Harrisonburg, VA (representative muni)
}

In [25]:
for year in range(2018,2026):
    df = pd.read_csv(getPjmFilePath(year))
    #print(df['zone'].unique())
    #print(df['load_area'].unique())
    #print(len(df['load_area'].unique()))
    set1 = set(df['load_area'].unique())
    set2 = set(zone_to_zips.keys())
    set2.add("RTO")
    # Note that years 2016 and 2017 do not match. Thus, we will start with 2018 inclusive
    if len(set1.symmetric_difference(set2)) > 0:
        print(str(year) + " did not match! The difference is: " + str(set1.symmetric_difference(set2)))
    
print("Finished checking inconsistencies!")
years = range(2018,2026)

Finished checking inconsistencies!


In [75]:
# Create a Nominatim geocoder instance for the desired country (For the USA, use 'us')
nomi = pgeocode.Nominatim('us')
years = range(2018,2026)

def getWeatherDf(load_area, force_refresh=False):
    # First check if we have the information cached
    file_path = getWeatherFilePath(load_area)
    if os.path.exists(file_path) and not force_refresh:
        temp = pd.read_csv(file_path)
        #print(temp.head())
        temp['time'] = pd.to_datetime(temp['time'])
        temp = temp.set_index('time')
        #print(temp.head())
        if temp.index.min().year == years[0] and temp.index.max().year == years[-1]:
            print("Cached weather file for " + load_area + " was found")
            return temp

    # Query the postal code
    zip_code = value[0]
    location_data = nomi.query_postal_code(zip_code)
    latitude = float(location_data.latitude)
    longitude = float(location_data.longitude)
    #print(latitude)
    #print(longitude)

    location = Point(latitude, longitude)
    start = datetime.datetime(years[0], 1, 1)
    end   = datetime.datetime(years[-1], 12, 31)

    data = Hourly(location, start, end).fetch()
    #print(data.head())
    data.to_csv(getWeatherFilePath(load_area))
    print("Added new cached weather file for " + load_area)
    return data

"""def getForecastWeatherDf(load_area, datetime_obj):
    # Query the postal code
    zip_code = value[0]
    location_data = nomi.query_postal_code(zip_code)
    latitude = float(location_data.latitude)
    longitude = float(location_data.longitude)
    location = Point(latitude, longitude)
    start = datetime_obj
    end   = datetime_obj + datetime.timedelta(days=1)
    data = Hourly(location, start, end).fetch()
    return data"""


for key, value in zone_to_zips.items():
    load_area = key
    getWeatherDf(load_area)

aeco_weather_df = getWeatherDf("AECO")
print(aeco_weather_df.head())

Cached weather file for AECO was found
Cached weather file for AEPAPT was found
Cached weather file for AEPIMP was found
Cached weather file for AEPKPT was found
Cached weather file for AEPOPT was found
Cached weather file for AP was found
Cached weather file for BC was found
Cached weather file for CE was found
Cached weather file for DAY was found
Cached weather file for DEOK was found
Cached weather file for DOM was found
Cached weather file for DPLCO was found
Cached weather file for DUQ was found
Cached weather file for EASTON was found
Cached weather file for EKPC was found
Cached weather file for JC was found
Cached weather file for ME was found
Cached weather file for OE was found
Cached weather file for OVEC was found
Cached weather file for PAPWR was found
Cached weather file for PE was found
Cached weather file for PEPCO was found
Cached weather file for PLCO was found
Cached weather file for PN was found
Cached weather file for PS was found
Cached weather file for RECO was 

In [26]:
import holidays
#from datetime import timedelta, date

us_holidays = holidays.US(years=years)

# Add black friday
for year in years:
    thanksgiving = [day for day, name in us_holidays.items() if name == "Thanksgiving Day" and day.year == year][0]
    us_holidays[thanksgiving + datetime.timedelta(days=1)] = "Black Friday"
    us_holidays[thanksgiving - datetime.timedelta(days=1)] = "Thanksgiving Eve"

#print(us_holidays)
#print(us_holidays.keys())

In [20]:
def isHoliday(datetime_obj):
     return datetime_obj.date() in us_holidays
def isThanksgiving(datetime_obj):
    return us_holidays.get(datetime_obj.date()) == "Thanksgiving Day"
def isThanksgivingEve(datetime_obj):
    return us_holidays.get(datetime_obj.date()) == "Thanksgiving Eve"
def isBlackFriday(datetime_obj):
    return us_holidays.get(datetime_obj.date()) == "Black Friday"
def isWeekend(datetime_obj):
    weekno = datetime.datetime.today().weekday()
    if weekno < 5:
        return False
    return True

# Thanksgiving 2024 (Nov 28, 2024)
#test_dt = datetime.datetime(2024, 11, 29, 15, 0)  # 3pm on Thanksgiving Day
#print(isHoliday(test_dt))
#print(isBlackFriday(test_dt))

In [43]:
# Load PJM data

def getEnergyDf(load_area):
    if load_area not in zone_to_zips.keys():
        print("Error: load_area(" + load_area + ") not found!")
        return -1

    ret_df = None
    for year in years:
        pjm_df = pd.read_csv(getPjmFilePath(year))
        pjm_df = pjm_df[pjm_df['load_area'] == load_area]
        
        if ret_df is None:
            ret_df = pjm_df
        else:
            ret_df = pd.concat([ret_df, pjm_df])

    ret_df['datetime_beginning_utc'] = pd.to_datetime(
        ret_df['datetime_beginning_utc'], 
        #utc=True
    )
    ret_df = ret_df.set_index('datetime_beginning_utc')
    ret_df = ret_df.sort_index()   # Always a good idea after setting index

    return ret_df

aeco_energy_df = getEnergyDf("AECO")
print(aeco_energy_df.head())

                       datetime_beginning_ept nerc_region mkt_region zone  \
datetime_beginning_utc                                                      
2018-01-01 05:00:00      1/1/2018 12:00:00 AM         RFC     MIDATL   AE   
2018-01-01 06:00:00       1/1/2018 1:00:00 AM         RFC     MIDATL   AE   
2018-01-01 07:00:00       1/1/2018 2:00:00 AM         RFC     MIDATL   AE   
2018-01-01 08:00:00       1/1/2018 3:00:00 AM         RFC     MIDATL   AE   
2018-01-01 09:00:00       1/1/2018 4:00:00 AM         RFC     MIDATL   AE   

                       load_area        mw  is_verified  
datetime_beginning_utc                                   
2018-01-01 05:00:00         AECO  1261.317         True  
2018-01-01 06:00:00         AECO  1221.330         True  
2018-01-01 07:00:00         AECO  1190.517         True  
2018-01-01 08:00:00         AECO  1174.050         True  
2018-01-01 09:00:00         AECO  1168.370         True  


In [81]:
# Last step is to augment the pjm data with:
# 1) weather at that current time  (we just use the ground truth weather)
# 2) previous day's weather
# 3) isHoliday, isThanksgiving, isThanksgivingEve, isBlackFriday, isWeekend
# 4) lagged loads (24 hrs ago, 168 hrs ago)

# The resulting dataframe could be viewed as X,y tuples
# in the sense that everything but 

# weather variables available:
# temp	dwpt	rhum	prcp	snow	wdir	wspd	wpgt	pres	tsun	coco

def add_features(energy_df, weather_df, dropna=True, outer=False):
    """
    df must contain at least:
    - load (float)
    - temp, dwpt, rhum, wspd, tsun, etc. (weather columns)
    - index is hourly timestamps (pd.DatetimeIndex)
    """

    """df = df.merge(
    weather_df[['temp']], # Only select 'temp', as the timestamp is the index
    left_on='datetime_beginning_utc',
    right_index=True,   # Tells pandas to use the index of weather_df (the DatetimeIndex)
    how='left'
    )"""
    df = energy_df.copy()

    if outer:
        df = df.join(weather_df[['temp']], how='outer')
    else:
        df = df.join(weather_df[['temp']], how='left')
    
    # --- Calendar Features ---
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['is_weekend'] = df['dayofweek'].isin([5,6]).astype(int)

    # Holiday features (assuming you already defined isHoliday / isThanksgiving):
    df['is_holiday'] = df.index.to_series().apply(isHoliday)
    df['is_thanksgiving'] = df.index.to_series().apply(isThanksgiving)
    df['is_thanksgiving_eve'] = df.index.to_series().apply(isThanksgivingEve)
    df['is_black_friday'] = df.index.to_series().apply(isBlackFriday)

    # --- Degree Days ---
    df['HDD'] = (18 - df['temp']).clip(lower=0)
    df['CDD'] = (df['temp'] - 18).clip(lower=0)

    # --- Rolling Weather Averages ---
    df['temp_rolling_24'] = df['temp'].rolling('24h', min_periods=1).mean()
    # df['dwpt_rolling_24'] = df['dwpt'].rolling(24, min_periods=1).mean()

    # --- Lagged Weather Features ---
    df['temp_lag_24'] = df['temp'].shift(freq='24h')
    df['temp_lag_168'] = df['temp'].shift(freq='168h')

    # --- Lagged Load Features (Key Predictors) ---
    df['mw_lag_24'] = df['mw'].shift(freq='24h')
    df['mw_lag_168'] = df['mw'].shift(freq='168h')

    # Drop rows where lag features are missing
    if dropna:
        df = df.dropna()

    return df

#df_augmented = add_features(aeco_energy_df, aeco_weather_df, False)
#print(df_augmented)

In [68]:
def getCompleteDf(load_area):
    # First check if we have the information cached
    file_path = getCompleteDfFilePath(load_area)
    if os.path.exists(file_path):
        temp = pd.read_csv(file_path)
        temp['datetime_beginning_utc'] = pd.to_datetime(temp['datetime_beginning_utc'])
        temp = temp.set_index('datetime_beginning_utc')
        temp = temp.sort_index()
        print("Cached complete file for " + load_area + " was found")
        return temp

    energy_df = getEnergyDf(load_area)
    weather_df = getWeatherDf(load_area)
    df_augmented = add_features(energy_df, weather_df)
    load_area_to_complete_df[load_area] = df_augmented
    df_augmented.to_csv(getCompleteDfFilePath(load_area))
    print("Added new cached complete file for " + load_area)
    return df_augmented

def getAllCompleteDfs():
    load_area_to_complete_df = {}
    for load_area in zone_to_zips.keys():
        load_area_to_complete_df[load_area] = getCompleteDf(load_area)

# print(getCompleteDf("AECO").head())

load_area_to_complete_df = getAllCompleteDfs()

Cached complete file for AECO was found
Cached complete file for AEPAPT was found
Cached complete file for AEPIMP was found
Cached complete file for AEPKPT was found
Cached complete file for AEPOPT was found
Cached complete file for AP was found
Cached complete file for BC was found
Cached complete file for CE was found
Cached complete file for DAY was found
Cached complete file for DEOK was found
Cached complete file for DOM was found
Cached complete file for DPLCO was found
Cached complete file for DUQ was found
Cached complete file for EASTON was found
Cached complete file for EKPC was found
Cached complete file for JC was found
Cached complete file for ME was found
Cached complete file for OE was found
Cached complete file for OVEC was found
Cached complete file for PAPWR was found
Cached complete file for PE was found
Cached complete file for PEPCO was found
Cached complete file for PLCO was found
Cached complete file for PN was found
Cached complete file for PS was found
Cached c

In [92]:
def getMidnightToday():
    now = datetime.datetime.now()
    return now.replace(hour=0, minute=0, second=0, microsecond=0)

def getMidnightTomorrow():
    tmrw = datetime.datetime.now() + datetime.timedelta(days=1)
    return tmrw.replace(hour=0, minute=0, second=0, microsecond=0)

def getPredictionFeatures(load_area):

    energy_df = getEnergyDf(load_area)
    weather_df = getWeatherDf(load_area, force_refresh=True)
    prediction_df = add_features(energy_df, weather_df, dropna=False, outer=True)

    start = getMidnightToday()
    end = getMidnightTomorrow()
    prediction_df = prediction_df.loc[start:end]
    prediction_df["load_area"] = load_area
    return prediction_df

# Note: The remaining issue is that the energy df that's given in canvas stops october 31. so we lose the mw predictors
aeco_prediction_features = getPredictionFeatures("AECO")
print(aeco_prediction_features)

Added new cached weather file for AECO
                    datetime_beginning_ept nerc_region mkt_region zone  \
2025-11-09 00:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 01:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 02:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 03:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 04:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 05:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 06:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 07:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 08:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 09:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 10:00:00                    NaN         NaN        NaN  NaN   
2025-11-09 11:00:00                    NaN         NaN        NaN  NaN   

In [87]:
print(getMidnightToday())
print(getMidnightTomorrow())

2025-11-09 00:00:00
2025-11-10 00:00:00
